# Lecture 2 and 3

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
# 3.0 LLMs and Chat Models 

from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI, ChatAnthropic
# from langchain.chat_models import ChatOllama

# llm = ChatOllama(model="llama3:latest")

llm = OpenAI(model_name='gpt-4o-mini')
chat = ChatOpenAI(model_name='gpt-4o-mini')

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a

/Users/hwangms/Documents/workspace/LLM_Study/langchain_study/hwang/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/hwangms/Documents/workspace/LLM_Study/langchain_study/hwang/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'As of now, there are eight recognized planets in our solar system. They are, in order from the Sun:\n\n1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n\nAdditionally, there are numerous dwarf planets, such as Pluto, Eris, and Haumea. In total, the number of known planets in the universe is vast, as astronomers have discovered thousands of exoplanets (planets outside our solar system) in various star systems. The exact number is constantly changing as new discoveries are made.'

In [7]:
# 3.1 Predicct Messages

from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.1)

In [9]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian.",
    ),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, What is your name?",)
]

chat.predict_messages(messages)

AIMessage(content='Ciao Paolo!\n\nIl mio nome è Geografia, e sono felice di aiutarti! La distanza tra Messico e Tailandia è di circa 18.000 chilometri.\n\nBuona giornata!')

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in {language}.",
    ),
    AIMessage(content="Ciao, mi chiamo {name}!"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, What is your name?",)
]

chat.predict_messages(messages)

## 3.2 Prompt Templates

In [15]:
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.1)

chat.predict(prompt)


template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, What is your name?",
        ),
    ]   
)

template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

[SystemMessage(content='You are a geography expert. And you only reply in Greek.'),
 AIMessage(content='Ciao, mi chiamo Socrates!'),
 HumanMessage(content='What is the distance between Mexico and Thailand. Also, What is your name?')]

## 3.3 OutputParser and LCEL(LangChain Expression Language, 표현언어)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.1)


In [21]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",") # 앞뒤 공백 제거 및 쉼표(,)기준으로 리스트로 변환 
        return list(map(str.strip,items))
    
    
p = CommaOutputParser()
p.parse("Hello, how, are, you")
    


['Hello', 'how', 'are', 'you']

In [28]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be\
            answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# 원본 소스 
# prompt = template.format_messages(max_items=10, question="What are the colors?")
# result = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(result.content)

# chain을 통해 코드 단축 가능 
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})


# chain을 여러개 둘 수 있음 
chain_one = template | chat | CommaOutputParser()
chain_two = template2 | chat | outputparser 

all = chain_one | chain_two | output

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})

['bulbasaur', 'charmander', 'squirtle', 'pikachu', 'charizard']

## 3.4 Chaining Chains

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "",
    ),
    ("human", "I want to cook {cuisine} food."),
    
])

chef_chain = chef_prompt | chat 

veg_chef_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a vegetarian chef specialized on making traditional recipies vegetarian.\
        You find alternative ingredients and explain their preparation.\
        You don't radically modify the recipe.\
        If there is no alternative for a food just say you don't know how to replace it.",
    ),
    ("human", "{recipe}"),    
])

veg_chain = veg_chef_prompt | chat 

final_chain = {"recipe": chef_chain} | veg_chain 

final_chain.invoke({
    "cuisine":"indian"
})


That sounds great! Indian cuisine is diverse and flavorful. Here are a few popular Indian dishes you might consider cooking, along with brief descriptions and basic recipes:

### 1. **Butter Chicken (Murgh Makhani)**
A rich and creamy chicken dish cooked in a spiced tomato sauce.

**Ingredients:**
- 500g chicken (boneless, cut into pieces)
- 1 cup yogurt
- 2 tablespoons butter
- 1 onion (finely chopped)
- 2 tomatoes (pureed)
- 1 tablespoon ginger-garlic paste
- 1 teaspoon garam masala
- 1 teaspoon chili powder
- 1 cup cream
- Salt to taste
- Fresh coriander for garnish

**Instructions:**
1. Marinate the chicken in yogurt, ginger-garlic paste, chili powder, and salt for at least 1 hour.
2. In a pan, melt butter and sauté onions until golden brown.
3. Add the marinated chicken and cook until browned.
4. Stir in the tomato puree and cook for 10 minutes.
5. Add cream and garam masala, simmer for another 10 minutes.
6. Garnish with coriander and serve with naan or rice.

### 2. **Chana Masa

AIMessageChunk(content='Here are vegetarian alternatives for the Indian dishes you mentioned, along with their preparation:\n\n### 1. **Vegetarian Butter Chicken (Murgh Makhani)**\nInstead of chicken, we can use **tofu** or **paneer** as the protein source.\n\n**Ingredients:**\n- 500g tofu or paneer (cubed)\n- 1 cup yogurt (use plant-based yogurt for vegan)\n- 2 tablespoons butter (or vegan butter)\n- 1 onion (finely chopped)\n- 2 tomatoes (pureed)\n- 1 tablespoon ginger-garlic paste\n- 1 teaspoon garam masala\n- 1 teaspoon chili powder\n- 1 cup cream (use coconut cream for vegan)\n- Salt to taste\n- Fresh coriander for garnish\n\n**Instructions:**\n1. Marinate the tofu or paneer in yogurt, ginger-garlic paste, chili powder, and salt for at least 1 hour.\n2. In a pan, melt butter and sauté onions until golden brown.\n3. Add the marinated tofu or paneer and cook until browned.\n4. Stir in the tomato puree and cook for 10 minutes.\n5. Add cream and garam masala, simmer for another 10 min

## 3.5 Recap

- 3장에서 배운 내용을 다시 정리 
- streaming기능 : response를 실시간으로 결과를 줌
- StreamingStdOutCallBackHandler() : 볼수 있는 문자(response)가 생길때마다 호출됨
- callback은 LLM에서 발생되는 다양한 이벤트들을 감지할 수 있음 